# Fine-tunen van Open AI-modellen

Dit notebook is gebaseerd op de huidige richtlijnen in de [Fine Tuning](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) documentatie van Open AI.

Fine-tuning verbetert de prestaties van foundation modellen voor jouw toepassing door het model opnieuw te trainen met extra data en context die relevant zijn voor dat specifieke gebruik of scenario. Houd er rekening mee dat prompt engineering technieken zoals _few shot learning_ en _retrieval augmented generation_ je in staat stellen om de standaardprompt te verrijken met relevante data om de kwaliteit te verbeteren. Deze methoden zijn echter beperkt door de maximale token window grootte van het gekozen foundation model.

Met fine-tuning trainen we het model zelf opnieuw met de benodigde data (waardoor we veel meer voorbeelden kunnen gebruiken dan in het maximale token window passen) en zetten we een _aangepaste_ versie van het model in die geen voorbeelden meer nodig heeft tijdens het uitvoeren van voorspellingen. Dit verbetert niet alleen de effectiviteit van ons prompt ontwerp (we hebben meer flexibiliteit om het token window voor andere zaken te gebruiken), maar kan ook de kosten verlagen (omdat we minder tokens naar het model hoeven te sturen bij het uitvoeren van voorspellingen).

Fine-tuning bestaat uit 4 stappen:
1. Bereid de trainingsdata voor en upload deze.
1. Start de training om een fijn-afgesteld model te krijgen.
1. Evalueer het fijn-afgestelde model en verbeter de kwaliteit indien nodig.
1. Zet het fijn-afgestelde model in voor voorspellingen zodra je tevreden bent.

Let op: niet alle foundation modellen ondersteunen fine-tuning - [controleer de OpenAI documentatie](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) voor de meest actuele informatie. Je kunt ook een eerder fijn-afgesteld model opnieuw fine-tunen. In deze tutorial gebruiken we `gpt-35-turbo` als ons foundation model voor fine-tuning.

---


### Stap 1.1: Bereid je dataset voor

Laten we een chatbot bouwen die je helpt het periodiek systeem van elementen te begrijpen door vragen over een element te beantwoorden met een limerick. In _deze_ eenvoudige tutorial maken we alleen een dataset om het model te trainen met een paar voorbeeldantwoorden die het verwachte formaat van de data laten zien. In een echte toepassing zou je een dataset met veel meer voorbeelden moeten maken. Mogelijk kun je ook een open dataset (voor jouw toepassingsgebied) gebruiken als die bestaat, en deze herformatteren voor gebruik bij fine-tuning.

Omdat we ons richten op `gpt-35-turbo` en op zoek zijn naar een enkelvoudig antwoord (chat completion), kunnen we voorbeelden maken met [dit voorgestelde formaat](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst) dat aansluit bij de OpenAI chat completion eisen. Als je verwacht dat het gesprek uit meerdere beurten bestaat, gebruik je het [multi-turn voorbeeldformaat](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst), waarin een `weight` parameter is opgenomen om aan te geven welke berichten wel of niet gebruikt moeten worden in het fine-tuning proces.

Voor deze tutorial gebruiken we het eenvoudigere single-turn formaat. De data staat in het [jsonl-formaat](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst) met 1 record per regel, elk als een JSON-geformatteerd object. Het fragment hieronder toont 2 records als voorbeeld – zie [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) voor de volledige voorbeeldset (10 voorbeelden) die we gebruiken voor onze fine-tuning tutorial. **Let op:** Elk record _moet_ op één regel staan (dus niet over meerdere regels verdeeld zoals gebruikelijk is in een opgemaakte JSON-file)

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

In een echte toepassing heb je een veel grotere set voorbeelden nodig voor goede resultaten – de afweging is tussen de kwaliteit van de antwoorden en de tijd/kosten voor fine-tuning. Wij gebruiken een kleine set zodat we het fine-tunen snel kunnen afronden om het proces te illustreren. Zie [dit OpenAI Cookbook voorbeeld](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) voor een meer uitgebreide fine-tuning tutorial.


### Stap 1.2 Upload je dataset

Upload de data met de Files API [zoals hier beschreven](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file). Let op dat je, om deze code te kunnen uitvoeren, eerst de volgende stappen moet hebben gedaan:
 - Het `openai` Python-pakket geïnstalleerd (zorg dat je versie >=0.28.0 gebruikt voor de nieuwste functies)
 - De omgevingsvariabele `OPENAI_API_KEY` ingesteld op je OpenAI API-sleutel
Meer informatie vind je in de [Setup-gids](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst) die bij de cursus hoort.

Voer nu de code uit om een bestand aan te maken voor upload vanuit je lokale JSONL-bestand.


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


### Stap 2.1: Maak de Fine-tuning-taak aan met de SDK



In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


### Stap 2.2: Controleer de status van de job

Hier zijn een paar dingen die je kunt doen met de `client.fine_tuning.jobs` API:
- `client.fine_tuning.jobs.list(limit=<n>)` - Toon de laatste n fine-tuning jobs
- `client.fine_tuning.jobs.retrieve(<job_id>)` - Haal details op van een specifieke fine-tuning job
- `client.fine_tuning.jobs.cancel(<job_id>)` - Annuleer een fine-tuning job
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - Toon tot n gebeurtenissen van de job
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

De eerste stap van het proces is _het valideren van het trainingsbestand_ om zeker te weten dat de data in het juiste formaat staat.


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


### Stap 2.3: Houd gebeurtenissen bij om de voortgang te volgen


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### Stap 2.4: Bekijk de status in het OpenAI-dashboard


Je kunt de status ook bekijken door naar de OpenAI-website te gaan en het gedeelte _Fine-tuning_ van het platform te openen. Hier zie je de status van de huidige taak en kun je ook de geschiedenis van eerdere uitvoeringen volgen. In deze screenshot zie je dat de vorige uitvoering is mislukt, en dat de tweede poging wel is geslaagd. Ter verduidelijking: dit gebeurde omdat de eerste poging een JSON-bestand gebruikte met verkeerd opgemaakte records – nadat dit was opgelost, is de tweede poging succesvol afgerond en werd het model beschikbaar gesteld voor gebruik.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-model-status.563271727bf7bfba7e3f73a201f8712fae3cea1c08f7c7f12ca469c06d234122.nl.png)


Je kunt de statusberichten en statistieken ook bekijken door verder naar beneden te scrollen in het visuele dashboard, zoals hieronder weergegeven:

| Berichten | Statistieken |
|:---|:---|
| ![Berichten](../../../../../translated_images/fine-tuned-messages-panel.4ed0c2da5ea1313b3a706a66f66bf5007c379cd9219cfb74cb30c0b04b90c4c8.nl.png) |  ![Statistieken](../../../../../translated_images/fine-tuned-metrics-panel.700d7e4995a652299584ab181536a6cfb67691a897a518b6c7a2aa0a17f1a30d.nl.png)|


### Stap 3.1: Haal ID op & Test Fijn-afgesteld Model in Code


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


### Stap 3.2: Laad & Test het Fijn-afgestelde Model in de Playground

Je kunt het fijn-afgestelde model nu op twee manieren testen. Ten eerste kun je naar de Playground gaan en via het Models-keuzemenu jouw nieuw fijn-afgestelde model selecteren uit de beschikbare opties. De andere mogelijkheid is om de optie "Playground" te gebruiken die wordt weergegeven in het Fine-tuning paneel (zie screenshot hierboven). Hiermee wordt de volgende _vergelijkende_ weergave gestart, waarin de foundation- en fijn-afgestelde modelversies naast elkaar worden getoond voor een snelle evaluatie.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-compare.56e06f0ad8922016497d39ced3d84ea296eec89073503f2bf346ec9718f913b5.nl.png)

Vul simpelweg de systeemcontext in die je in je trainingsdata hebt gebruikt en geef je testvraag op. Je zult merken dat beide zijden worden bijgewerkt met dezelfde context en vraag. Voer de vergelijking uit en je ziet het verschil in de uitkomsten tussen beide modellen. _Let op hoe het fijn-afgestelde model het antwoord weergeeft in het formaat dat je in je voorbeelden hebt opgegeven, terwijl het foundation model simpelweg de systeem prompt volgt_.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-launch.5a26495c983c6350c227e05700a47a89002d132949a56fa4ff37f266ebe997b2.nl.png)

Je zult merken dat de vergelijking ook het aantal tokens voor elk model toont, evenals de tijd die nodig is voor de inferentie. **Dit specifieke voorbeeld is een eenvoudig voorbeeld om het proces te laten zien, maar weerspiegelt geen echte dataset of scenario**. Je zult zien dat beide voorbeelden hetzelfde aantal tokens tonen (systeemcontext en gebruikersprompt zijn identiek), waarbij het fijn-afgestelde model meer tijd nodig heeft voor de inferentie (aangepast model).

In echte situaties gebruik je geen speelgoedvoorbeeld als dit, maar ga je fijn-afstellen op echte data (bijvoorbeeld een productcatalogus voor klantenservice), waarbij de kwaliteit van het antwoord veel duidelijker zal zijn. In _die_ context zal het verkrijgen van een gelijkwaardige antwoordkwaliteit met het foundation model meer maatwerk in prompt engineering vereisen, wat het tokengebruik en mogelijk de verwerkingstijd voor inferentie zal verhogen. _Om dit zelf te proberen, bekijk de fine-tuning voorbeelden in het OpenAI Cookbook om te beginnen._



---

**Disclaimer**:  
Dit document is vertaald met behulp van de AI-vertalingsdienst [Co-op Translator](https://github.com/Azure/co-op-translator). Hoewel we streven naar nauwkeurigheid, dient u er rekening mee te houden dat geautomatiseerde vertalingen fouten of onnauwkeurigheden kunnen bevatten. Het originele document in de oorspronkelijke taal moet als de gezaghebbende bron worden beschouwd. Voor kritische informatie wordt professionele menselijke vertaling aanbevolen. Wij zijn niet aansprakelijk voor misverstanden of verkeerde interpretaties die voortvloeien uit het gebruik van deze vertaling.
